# Visualization
When it comes to visualizing geospatial data with/on maps with Python, a great number of tools and techniques
are available. In this lesson we will explore several of these:

* [Folium](https://github.com/python-visualization/folium)
* [ipyleaflet](https://ipyleaflet.readthedocs.io) - Interactive maps in the Jupyter notebook
* [Bokeh](https://bokeh.pydata.org)


## Folium
Whenever you visit website that has some kind of interactive map, it
is quite probable that you are witnessing a map that has been made with
a JavaScript library called [Leaflet](http://leafletjs.com).  The
other popular library one that you might encounter is
[OpenLayers](https://openlayers.org).

There is a Python module called
[Folium](https://github.com/python-visualization/folium) that makes
it possible to visualize data that has been manipulated in Python on an
interactive Leaflet map.

### Basics
We will start with the most minimal map using the default OpenStreetMap base map.
See [Folium Quickstart](https://python-visualization.github.io/folium/quickstart.html).


In [1]:
import folium

folium_map = folium.Map(location=[44.43225, 26.10626])

To display it in a Jupyter notebook, simply ask for the object representation:


In [2]:
folium_map

You could even save this map to a file and serve it via a webserver: 


In [114]:
folium_map.save('test/07-folium-1.html')


Now [open this map here](test/07-folium-1.html) as "regular" HTML! 


### GeoJSON overlay
It gets interesting when you can overlay the map with data manipulated
via Python. Here we overlay the map with the Polygons of all countries, though
that set is in a lower resolution clearly.


In [115]:

countries = f'../data/countries.json'

folium_map2 = folium.Map(
    location=[44.43225, 26.10626],
    zoom_start=2  
)

folium.GeoJson(
    countries,
    name='countries'
).add_to(folium_map2)

folium.LayerControl().add_to(folium_map2)

folium_map2


In [116]:
# Also save this map
folium_map2.save('test/07-folium-2.html')


And [open this map with overlay here](test/07-folium-2.html). 

Much more is possible with Folium, we just scratched the surface here!

## Interactive maps in the Jupyter notebook with ipyleaflet
ipyleaflet is a Jupyter/Leaflet bridge enabling interactive maps in the Jupyter notebook.
(Although one can always save the results and use these maps in non-Jupyter contexts!).

`ipyleaflet` is based on [ipywidgets](https://ipywidgets.readthedocs.io).
ipywidgets are interactive HTML widgets for Jupyter notebooks and the IPython kernel.

Links:

* GitHub: https://github.com/jupyter-widgets/ipyleaflet
* Documentation: https://ipyleaflet.readthedocs.io

### ipyleaflet - simplest map

In [ ]:
from ipyleaflet import *

m = Map(center=(44.4, 26.1), zoom=4, basemap=basemaps.Hydda.Full)
m

### ipyleaflet - add overlay tiles


In [ ]:
strata_all = basemap_to_tiles(basemaps.Strava.All)
m.add_layer(strata_all)
m

### ipyleaflet - mouse interaction handling
We'll remove the overlay and add interaction to show lat/lon coordinates where your mouse is. 

In [ ]:
m.remove_layer(strata_all)

from ipywidgets import Label

label = Label()
display(label)

def handle_interaction(**kwargs):
    if kwargs.get('type') == 'mousemove':
        label.value = str(kwargs.get('coordinates'))

m.on_interaction(handle_interaction)
m

### ipyleaflet - Add Overlay Vector Layer
We will add some rivers.


In [ ]:
import json

with open('../data/rivers_lake_centerlines.json') as f:
    data = json.load(f)
    
for feature in data['features']:
    feature['properties']['style'] = {
        'color': 'blue',
        'weight': 1,
        'fillColor': 'blue',
        'fillOpacity': 0.5
    }
geo = GeoJSON(data=data, hover_style={'fillColor': 'red'}, name='Rivers-Lakes')
m.add_layer(geo)
m

### ipyleaflet - Adding Control
Add a control to select layers to display on map.


In [ ]:
m.add_control(LayersControl())

### ipyleaflet - Creating two maps side by side
As ipyleaflet is based on ipywidgets, we can make interesting combinations.


In [ ]:
import ipywidgets
 
ipywidgets.HBox([m, Map(center=[44.4, 26.1], zoom=8)])

## Bokeh

Bokeh is a very powerful framework to produce powerful maps in combination
with data. With Geopandas and Bokeh one can produce nice looking interactive maps like in the image below:

![Bokeh and Geopandas Example](images/bokeh-example1.jpg)
*Interactive Map with Bokeh and GeoPandas - Source: [CSC L6](https://automating-gis-processes.github.io/CSC/lessons/L6/interactive-map-bokeh.html)*


### Bokeh - links

See also:

* https://automating-gis-processes.github.io/CSC/lessons/L6/interactive-map-bokeh.html
* [Binder for Geographic Plots in Bokeh](https://mybinder.org/v2/gh/bokeh/bokeh-notebooks/master?filepath=tutorial%2F09%20-%20Geographic%20Plots.ipynb)
* https://towardsdatascience.com/exploring-and-visualizing-chicago-transit-data-using-pandas-and-bokeh-part-ii-intro-to-bokeh-5dca6c5ced10
* https://pythonawesome.com/bokeh-plotting-backend-for-pandas-and-geopandas/


### Bokeh - making a simple plot
First, we learn the basic logic of plotting in Bokeh by making a simple interactive plot with a few points.

Import the necessary functionalities from Bokeh.

In [1]:
from bokeh.plotting import figure, save

Initialize our plot by calling the `figure` object.


In [2]:
p = figure(title="My first interactive plot!")

Next we create lists of x and y coordinates that we want to plot.


In [3]:
x_coords = [0,1,2,3,4]
y_coords = [5,4,1,2,0]

	In Bokeh drawing points, lines or polygons are always done using 
	list(s) of x and y coordinates.

Now we can plot those as points using a `.circle()` -object. Give it a red color and size of 10.


In [4]:
p.circle(x=x_coords, y=y_coords, size=10, color="red")

GlyphRenderer(id='1038', ...)

Finally, we can save our interactive plot into the disk with save -function 
that we imported in the beginning. All interactive plots are typically 
saved as html files which you can open in a web-browser.
	
	# Save the plot by passing the plot -object and output path
	save(obj=p, filename="../data/output/07-points.html")

Now you could open your interactive `points.html` plot by double-clicking it which should open it in a web browser.

But we will plot directly in the Notebook here using `output_notebook()` and `show()`.


In [5]:
from bokeh.io import output_notebook, show
output_notebook()

Loading BokehJS ...

And then the moment of magic:


In [6]:
show(p)

### Bokeh - Creating an Interactive Tiled Background Map


In [7]:
from bokeh.plotting import figure
from bokeh.tile_providers import get_provider, Vendors
from bokeh.io import output_notebook, show
output_notebook()

Loading BokehJS ...

If you show the figure, you can then use the wheel zoom and pan tools to navigate over any zoom level, 
and Bokeh will request the appropriate tiles from the server and insert them at the correct locations in the plot:


In [8]:
# When using in standard Python env
# output_file("tile.html")

tile_provider = get_provider(Vendors.CARTODBPOSITRON)

# range bounds supplied in web mercator coordinates
p = figure(tools='pan, wheel_zoom', x_range=(-2000000, 6000000), y_range=(-1000000, 7000000),
           x_axis_type="mercator", y_axis_type="mercator")
p.add_tile(tile_provider)

show(p)

### Creating an Interactive Maps using Bokeh and Geopandas

Creating an interactive Bokeh map from a Shapefile or other vector data file like GeoJSON
consists typically of the following steps:

* Read the spatial vector file into `GeoDataFrame`
* Calculate the x and y coordinates of the geometries into separate columns
* Convert the `GeoDataFrame` into a Bokeh `DataSource`
* Plot the x and y coordinates as points, lines or polygons (which are in Bokeh words: `circle`, `multi_line` and `patches`)

We follow the steps below, extending and plotting on the tiled map from above.


In [ ]:
import geopandas as gpd

# Read the data (already in Web Mercator projection
points = gpd.read_file('../data/populated_places.3857.gpkg')

In [ ]:
def getPointCoords(row, geom, coord_type):
    """Calculates coordinates ('x' or 'y') of a Point geometry"""
    if coord_type == 'x':
        return row[geom].x
    elif coord_type == 'y':
        return row[geom].y

In [ ]:
points['x'] = points.apply(getPointCoords, geom='geometry', coord_type='x', axis=1)
points['y'] = points.apply(getPointCoords, geom='geometry', coord_type='y', axis=1)

In [ ]:
points.head(5)

In [ ]:
p_df = points.drop('geometry', axis=1).copy()
p_df.head(2)

In [ ]:
from bokeh.models import ColumnDataSource
psource = ColumnDataSource(p_df)

In [ ]:
# p = figure(title="A map of populated places from a GeoPackage")
p.circle('x', 'y', source=psource, color='lightblue',size=15, alpha=0.7)
show(p)

### Using Bokeh GeoJSONSource
The above scenario could be effected even more compact.
The Bokeh `GeoJSONDataSource` expects a GeoJSON-string, so we can 
just use ordinary file `open()/read()`.

See `GeoJSONDataSource` example in [Bokeh Documentation](https://bokeh.pydata.org/en/latest/docs/user_guide/geo.html#geojson-data). 
Though we have to do any reprojection from WGS84 (default for GeoJSON)
to EPSG:3857 (Web Mercator) before. In this case we reprojected
the GeoJSON file before using `ogr2ogr`:

	ogr2ogr -s_srs EPSG:4326 -t_srs EPSG:3857 populated_places.3857.json populated_places.json


In [ ]:
from bokeh.models import GeoJSONDataSource
from bokeh.plotting import figure
from bokeh.tile_providers import get_provider, Vendors
from bokeh.io import output_notebook, show
output_notebook()

# We could also assemble map plus overlay in an HTML file
# output_file("../data/output/07-bokeh-geojson.html")

Read the populated places GeoJSON


In [ ]:
with open('../data/populated_places.3857.json') as json_file:
        geojson = json_file.read()
        
geo_source = GeoJSONDataSource(geojson=geojson)

Build the map. Again, use the wheel zoom and pan tools to navigate the map. 


In [ ]:
p = figure(tools='pan, wheel_zoom', x_axis_type="mercator", y_axis_type="mercator", width=800, height=500)

# add background tiles layer from CARTO
p.add_tile(get_provider(Vendors.CARTODBPOSITRON))

# add populated places point overlay
p.circle(x='x', y='y', size=10, alpha=0.7, source=geo_source, color='lightblue', legend="Populated Places")

show(p)

---
[<- Data analysis](06-data-analysis.ipynb) | [Metadata ->](08-metadata.ipynb)